## Importing Dependencies

In [1]:
import scipy.io
import numpy as np
from random import shuffle
import random
#import spectral
import scipy.ndimage
#from skimage.transform import rotate
import os
# import patch_size
patch_size = 21 
%matplotlib inline

## Loading Indian Pines Data (Numpy Array)

In [2]:
DATA_PATH = os.path.join(os.getcwd(),"IndianPines")
NEW_DATA_PATH = os.path.join(os.getcwd(),"new_indian_pines_data")
input_mat = scipy.io.loadmat(os.path.join(DATA_PATH, 'Indian_pines_corrected.mat'))['indian_pines_corrected']
target_mat = scipy.io.loadmat(os.path.join(DATA_PATH, 'Indian_pines_gt.mat'))['indian_pines_gt']

print("Input Matrix Type: %s" %type(input_mat))
print("Target Matrix Type: %s" %type(input_mat))

print("\n")
print("Input Matrix: %s" %str(np.shape(input_mat)))
print("Target Matrix: %s" %str(np.shape(input_mat)))

Input Matrix Type: <class 'numpy.ndarray'>
Target Matrix Type: <class 'numpy.ndarray'>


Input Matrix: (145, 145, 200)
Target Matrix: (145, 145, 200)


In [3]:
# Now let us define our global variables that define our data
HEIGHT = input_mat.shape[0]
WIDTH = input_mat.shape[1]
BAND = input_mat.shape[2]
print(BAND)
PATCH_SIZE = patch_size
TRAIN_PATCH, TRAIN_LABELS, TEST_PATCH, TEST_LABELS, CLASSES = [],[],[],[],[]
COUNT =200 # Number of patches of each class (This will change depending on the selected patch size)
OUTPUT_CLASSES = 16
TEST_FRAC = 0.25 # percentage used for testing


200


## Scaling input between [0,1]

In [4]:
input_mat =input_mat.astype(float) # change elements in the tensor to float
input_mat -= np.min(input_mat) # shift [0 to max]
input_mat /= np.max(input_mat) # scale ([0 to max])/max = [0 to 1] array

## Normalize data

In [5]:
# calculate the mean of each channel for normalization
MEAN_ARRAY = np.ndarray(shape=(BAND,),dtype=float) #row vector containing the mean of each batch
for i in range(BAND):
    MEAN_ARRAY[i] = np.mean(input_mat[:,:,i]) # mean per band
    print(MEAN_ARRAY[i])
#np.transpose(input_mat, (2,0,1)).shape transpose

0.2315138712055816
0.36262241144907714
0.38414871768010406
0.37171426414963604
0.4118023665455059
0.4434726729833022
0.45014655732642966
0.43470141159879233
0.42940280120085644
0.40282871326426084
0.3991435518859513
0.38762190208733827
0.3692214904185689
0.39836681991512285
0.4065162227933123
0.4038283160858362
0.3851393293390024
0.3601987789341183
0.3482859173233721
0.3360189028884316
0.3328332597130334
0.3301365213191603
0.3222580796388797
0.3124842458744792
0.31177440595429434
0.30145266118480707
0.2888512359892871
0.2929874127846909
0.2903191161604601
0.26543948019531444
0.2947900226690033
0.26239219094149985
0.29558751955131074
0.34611657248629984
0.3721408796959062
0.4035176562925973
0.5034966576658805
0.5776345735776125
0.576080559717749
0.36797551324209915
0.5941163701163998
0.6079502390013266
0.5780108826063484
0.5704550999345734
0.5223399349639233
0.4694857893574055
0.5047177455443221
0.5541423757483872
0.5451722254461178
0.5462589738058836
0.5382964551310049
0.54202989382866

## Patching function

In [6]:
def patch(height_index, width_index):
    """
    Returns a mean-normalized patch, the top left corner of which 
    is at (height_index, width_index)
    
    Inputs: 
    height_index - row index of the top left corner of the image patch
    width_index - column index of the top left corner of the image patch
    
    Outputs:
    mean_normalized_patch - mean normalized patch of size (PATCH_SIZE, PATCH_SIZE) 
    whose top left corner is at (height_index, width_index)
    """
    
    
    
    # creating patch slices
    transposed_array = np.transpose(input_mat, (2,0,1))
    height_slice = slice(height_index, height_index+PATCH_SIZE) # patch size is similar to the kernel size (same equations and logic apply)
    width_slice = slice(width_index, width_index+PATCH_SIZE)
    transposed_patch = transposed_array[:,height_slice,width_slice] # include all of the tensor's depth
    mean_normalized_patch = []
    for i in range(transposed_patch.shape[0]): # for every band normalize patch
        mean_normalized_patch.append(transposed_patch[i]-MEAN_ARRAY[i]) # substract the mean for every element of in the matrix
    return np.array(mean_normalized_patch)

## Obtain all patches and separate them by class 

In [7]:
CLASSES = []
for classes in range(OUTPUT_CLASSES):
    CLASSES.append([]) # append the number of list classes 
    
    
# getting patches

for i in range(HEIGHT - PATCH_SIZE +1):
    for j in range(WIDTH - PATCH_SIZE +1):
        # the actual patch tensor
        input_img = patch(i,j)
        #choose the middle of the patch as classification
        target_id = target_mat[i+int((PATCH_SIZE-1)/2),j+int((PATCH_SIZE-1)/2)]
        if(target_id != 0): # ignore the UNKNOWN patches
            CLASSES[target_id -1].append(input_img)
            
print("Done")
print(type(CLASSES[0]))

Done
<class 'list'>


In [8]:
print(input_mat.shape)
print(target_mat.shape)
print(len(CLASSES))
print(np.array(CLASSES[0]).shape)
print("\n")

for c in range(len(CLASSES)):
    print("Class %d contains: %s patches"%(c+1,len(CLASSES[c])))

(145, 145, 200)
(145, 145)
16
(46, 200, 21, 21)


Class 1 contains: 46 patches
Class 2 contains: 1378 patches
Class 3 contains: 547 patches
Class 4 contains: 152 patches
Class 5 contains: 340 patches
Class 6 contains: 730 patches
Class 7 contains: 28 patches
Class 8 contains: 356 patches
Class 9 contains: 20 patches
Class 10 contains: 843 patches
Class 11 contains: 2245 patches
Class 12 contains: 481 patches
Class 13 contains: 205 patches
Class 14 contains: 1095 patches
Class 15 contains: 142 patches
Class 16 contains: 93 patches


## Extract 25% from data set for testing

In [9]:
for i in range(len(CLASSES)):
    class_population = len(CLASSES[i])
    split_size = int(class_population*TEST_FRAC) # getting 25% (spliting size for the test)
    patches_class = CLASSES[i] # extracting patches from class
    shuffle(patches_class) # shuffling patches
    TRAIN_PATCH.append(patches_class[:-split_size]) # from 0 to (len -split_size) contains patches
    TEST_PATCH.extend(patches_class[-split_size:]) # from split_size to len(class)-1
    TEST_LABELS.extend(np.full(split_size,i,dtype=int)) # 0 to 15 label

for c in TRAIN_PATCH:
    print(len(c))
print("____________")

print(len(TRAIN_PATCH))
print(len(TEST_PATCH))
print("____________")
print(np.shape(TRAIN_PATCH[0]))
print(np.shape(TEST_PATCH[0]))
    

35
1034
411
114
255
548
21
267
15
633
1684
361
154
822
107
70
____________
16
2170
____________
(35, 200, 21, 21)
(200, 21, 21)


In [10]:
#x = CLASSES[0]
#print(int(len(x)*TEST_FRAC))
#print(np.shape(x))
#np.shape(x[:-int(class_population*TEST_FRAC)])
#np.full(int(len(x)*TEST_FRAC),len(x))

## Oversample the classes which do not have at least COUNT patches in the training set and extract COUNT patches

In [11]:
# This methods ensures that there are at least COUNT samples for every class
for i in range(len(TRAIN_PATCH)):
    if(len(TRAIN_PATCH[i])<COUNT):
        temp = TRAIN_PATCH[i]
        for j in range(int(COUNT/len(TRAIN_PATCH[i]))):
            shuffle(TRAIN_PATCH[i])
            TRAIN_PATCH[i] = TRAIN_PATCH[i]+temp # adding oversamples to the TRAININ_PATCH
    shuffle(TRAIN_PATCH[i])
    TRAIN_PATCH[i] = TRAIN_PATCH[i][:COUNT] # include all new oversamples of at most COUNT
            

In [12]:
# Checking the code from above
for c in range(len(TRAIN_PATCH)):
    print(len(TRAIN_PATCH[c]))

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


## Convert data to numpy arrays

In [13]:
TRAIN_PATCH = np.asarray(TRAIN_PATCH)
print(np.shape(TRAIN_PATCH))
TRAIN_PATCH = TRAIN_PATCH.reshape((-1,200,PATCH_SIZE,PATCH_SIZE)) # reduce dimension of the training tensor
print(np.shape(TRAIN_PATCH))

(16, 200, 200, 21, 21)
(3200, 200, 21, 21)


## Training and Testing Labels

In [14]:
TRAINING_LABELS = np.array([])
for i in range(OUTPUT_CLASSES):
    TRAINING_LABELS = np.append(TRAINING_LABELS,np.full(COUNT,i,dtype=int))
print("The length of training labels is: %d"%len(TRAINING_LABELS))
print("The length of testing labels is: %d"%len(TEST_LABELS))

The length of training labels is: 3200
The length of testing labels is: 2170


## Saving Training and Testing Data

In [15]:
l_train = int(len(TRAIN_PATCH)/(COUNT)) # 32000/200
l_testing = int(len(TEST_PATCH)/(COUNT))
for i in range(l_train):
    train_dict ={}
    start = i*COUNT
    end = (i+1)*COUNT
    train_dict["train_patches"] = TRAIN_PATCH[start:end]
    train_dict["train_labels"] = TRAINING_LABELS[start:end]
    file_name = "Training(%s)_class(%d).mat"%(str(PATCH_SIZE),i)
    print(len(TRAINING_LABELS[start:end]))
    scipy.io.savemat(os.path.join(NEW_DATA_PATH,file_name),train_dict)

for i in range(l_testing):
    test_dict = {}
    start = i*COUNT
    end = (i+1)*COUNT
    test_dict["testing_patches"] = TEST_PATCH[start:end]
    test_dict["testing_labels"] = TEST_LABELS[start:end]
    file_name = "Testing(%s)_class(%d).mat"%(str(PATCH_SIZE),i)
    scipy.io.savemat(os.path.join(NEW_DATA_PATH,file_name),test_dict)
print(train_dict["train_labels"])
print('Done')




200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
[15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15.
 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15.
 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15.
 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15.
 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15.
 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15.
 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15.
 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15.
 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15.
 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15.
 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15. 15.
 15. 15.]
Done


In [16]:
print(l_train)
print(l_testing)
print(np.shape(train_dict["train_patches"]))
print(np.shape(train_dict["train_labels"]))
print(np.shape(TRAINING_LABELS))
print(TRAINING_LABELS)


16
10
(200, 200, 21, 21)
(200,)
(3200,)
[ 0.  0.  0. ... 15. 15. 15.]


In [19]:
list_of_dicts = []
for files in os.listdir(NEW_DATA_PATH):
    temp = scipy.io.loadmat(os.path.join(NEW_DATA_PATH,files))
    list_of_dicts.append(temp)
print(np.shape(list_of_dicts[0]["testing_patches"]))
print(type((list_of_dicts[0]["testing_labels"][0][0])))

(200, 200, 21, 21)
<class 'numpy.int64'>
